Let's try to look at all samples to see if there is anything there, as there wasn't much to go with in the simplex only. True, maybe looking at ADHD yes/no is not the best way to go about this. Maybe looking at brain metrics or some graph theory idea would be better. But we need to do this bit anyways, so let's go with it in parallel. 

# New samples

The first step is to make sure we have all files for all participants. I created the file all_samples.ped in data/cnv, derived from pedigree_20171205.csv. Then, I replaced all IDs by their WES code. Now, we run this code to populate our trio dictionary:

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

%matplotlib inline

In [ ]:
# figure out who is who in each trio
import glob
data_dir = '/data/NCR_SBRB/cnvsimplex/'
trios = {}
affected = []
controls = []
peds = glob.glob(data_dir + '*trio*ped')
for ped in peds:
    trio_name = ped.split('/')[-1].split('.')[0]
    fid = open(ped, 'r')
    fam = {}
    for line in fid:
        famid, sid, fa, mo, sex, aff = line.rstrip().split('\t')
        if fa != '0':
            fam['child'] = sid
            if aff == '1':
                affected.append(trio_name)
            else:
                controls.append(trio_name)
        elif sex == '1':
            fam['father'] = sid
        else:
            fam['mother'] = sid
    trios[trio_name] = fam
    fid.close()
    
fid = open('/data/NCR_SBRB/simplex/famids.txt', 'r')
fams = [t.rstrip() for t in fid]
fid.close()

# CCGO_800976 was removed during sample filtering step in XHMM, let's remove the family that has that sample, as it's a father
del trios['10369_trio1']
del trios['10369_trio2']
fams.remove('10369')

# PennCNV

Let's run the joint calling algorithm for all samples. It takes longer, but supposedly it's more accurate than the trio/quartet calling (http://penncnv.openbioinformatics.org/en/latest/user-guide/joint/). The drawback is that it only works for trios, but we were planning on working with trios only anyways.

